#### Bibliotecas

In [1]:
import pandas as pd
from google.cloud import bigquery
from platform import python_version
from google.oauth2 import service_account
from datetime import datetime
import gspread as gs
from oauth2client.service_account import ServiceAccountCredentials
import calendar

In [2]:
# Versão do notebook utilizada
print("Versão utilizada deste notebook:", python_version())

Versão utilizada deste notebook: 3.12.4


In [4]:
# Variáveis de tempo

data_ini = '2020-01-01'
data_atual = datetime.now().date()

#### Criação de Consultas

In [5]:
qry_vendas = f"""
SELECT 
  EXTRACT(YEAR FROM o.created_at) AS Ano,
  EXTRACT(MONTH FROM o.created_at) AS Mes,
  o.order_id AS ID_Pedido,
  o.user_id AS ID_Cliente,
  CASE
    WHEN o.status = "Shipped"
    THEN "Enviado"
    ELSE "Processando"
    END AS Status_Pedido,
  DATE(o.created_at) AS Data_Pedido,
  CASE 
    WHEN DATE(o.shipped_at) is null
    THEN '1900-01-01'
    ELSE DATE(o.shipped_at)
    END AS Data_Envio,
  ROUND(SUM(oi.sale_price),2) AS Valor_Pedido,
  o.num_of_item AS Quantidade_Itens,
  u.state AS Estado,
  CASE
    WHEN u.city is null
    THEN 'Outra'
    ELSE u.city
    END AS Cidade,
  u.country AS Pais,
  u.postal_code
FROM bigquery-public-data.thelook_ecommerce.orders o
INNER JOIN bigquery-public-data.thelook_ecommerce.users u
  ON u.id = o.user_id
INNER JOIN bigquery-public-data.thelook_ecommerce.order_items oi
  ON oi.order_id = o.order_id
WHERE 
  o.status IN ('Shipped', 'Processing') AND
  DATE(o.created_at) BETWEEN '{data_ini}' AND '{data_atual}' AND
  u.country = "Brasil"
GROUP BY 1,2,3,4,5,6,7,9,10,11,12,13
ORDER BY 6 DESC


"""

In [6]:
qry_pub = f""" WITH PUB AS (SELECT
  EXTRACT(YEAR FROM e.created_at) AS Ano,
  EXTRACT(MONTH FROM e.created_at) AS Mes,
  DATE(e.created_at) AS Data_Acesso,
  COUNT(e.session_id) AS ID,
  e.city AS Cidade,
  e.state AS Estado,    
  e.traffic_source AS Origem,
  e.event_type AS Tipo_Evento,
  e.postal_code AS CEP
FROM bigquery-public-data.thelook_ecommerce.events e
INNER JOIN bigquery-public-data.thelook_ecommerce.users u
  ON e.postal_code= u.postal_code AND e.city = u.city
WHERE
  DATE(e.created_at) BETWEEN '{data_ini}' AND '{data_atual}' AND
  u.country = "Brasil"
GROUP BY 1,2,3,5,6,7,8,9
ORDER BY 1 DESC)

SELECT * FROM PUB

"""

In [7]:
# CONSULTA DE CLIENTES

qry_clientes = f"""
WITH CLIENTES AS(
SELECT
  DATE(u.created_at) AS Data_Cadastro,
  u.id AS ID_Cliente,
  u.age AS Idade,
  CASE
    WHEN u.gender = "F" THEN 'Feminino'
    WHEN u.gender = 'M' THEN 'Masculino'
    ELSE 'Outro'
  END AS Genero,
  u.postal_code AS CEP
FROM bigquery-public-data.thelook_ecommerce.users u
WHERE DATE(u.created_at) BETWEEN '{data_ini}' AND '{data_atual}')

SELECT * FROM CLIENTES

"""

#### Conexão e Extração de Dados

In [8]:
# Configurando credenciais
credentials = service_account.Credentials.from_service_account_file(filename = 'C:/Users/User/Documents/Projeto_Google_Big_Query/gbq.json',
                                                                    scopes = ['https://www.googleapis.com/auth/cloud-platform'])

In [9]:
# gerando dataframe
df_vendas = pd.read_gbq(credentials=credentials, query = qry_vendas)
df_pub = pd.read_gbq(credentials=credentials, query = qry_pub)
df_cliente = pd.read_gbq(credentials=credentials, query = qry_clientes)

C:\Users\User\AppData\Local\Temp\ipykernel_10332\3412748997.py:2: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vendas = pd.read_gbq(credentials=credentials, query = qry_vendas)
C:\Users\User\AppData\Local\Temp\ipykernel_10332\3412748997.py:3: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_pub = pd.read_gbq(credentials=credentials, query = qry_pub)
C:\Users\User\AppData\Local\Temp\ipykernel_10332\3412748997.py:4: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_cliente = pd.read_gbq(credentials=credentials, query = qry_clientes)


#### Análise exploratória dos Dados

In [10]:
# dados da coluna "Data_Envio" com valor de "1900-01-01" denota que não houve envio. A data foi utilizada para normalizar os valores na coluna, sem ausência de informação (nulo).
df_vendas.head(10)

,Ano,Mes,ID_Pedido,ID_Cliente,Status_Pedido,Data_Pedido,Data_Envio,Valor_Pedido,Quantidade_Itens,Estado,Cidade,Pais,postal_code
0,2024,6,29236,23430,Processando,2024-06-29,1900-01-01,24.00,1,Paraná,Jacarezinho,Brasil,86400-000
1,2024,6,33232,26611,Processando,2024-06-29,1900-01-01,36.99,2,Rio de Janeiro,Casimiro de Abreu,Brasil,28860-000
2,2024,6,37334,29917,Processando,2024-06-29,1900-01-01,182.50,2,Mato Grosso do Sul,Três Lagoas,Brasil,79640-310
3,2024,6,80032,64140,Processando,2024-06-29,1900-01-01,91.46,2,Ceará,Fortaleza,Brasil,60714-222
4,2024,6,111612,89363,Processando,2024-06-29,1900-01-01,9.20,1,Bahia,Camaçari,Brasil,42800-970
5,2024,6,119922,96076,Processando,2024-06-29,1900-01-01,233.75,3,Pará,null,Brasil,68695-000
6,2024,6,254,204,Enviado,2024-06-29,2024-06-30,214.14,4,Bahia,Catu,Brasil,48110-000
7,2024,6,1796,1444,Enviado,2024-06-29,2024-06-30,19.99,1,Bahia,Dias d'Ávila,Brasil,42850-000
8,2024,6,6899,5548,Enviado,2024-06-29,2024-06-30,12.99,1,São Paulo,Santa Cruz do Rio Pardo,Brasil,18900-000
9,2024,6,9399,7527,Enviado,2024-06-29,2024-07-01,33.00,1,Paraná,Curitiba,Brasil,82590-300


In [11]:
# Verificando os tipos de dados de cada coluna para manipulação adequada.

df_vendas.dtypes

Ano                   Int64
Mes                   Int64
ID_Pedido             Int64
ID_Cliente            Int64
Status_Pedido        object
Data_Pedido          dbdate
Data_Envio           dbdate
Valor_Pedido        float64
Quantidade_Itens      Int64
Estado               object
Cidade               object
Pais                 object
postal_code          object
dtype: object

In [16]:
# Total de Vendas
print(f"O total de vendas do período é de:", round(df_vendas['Valor_Pedido'].sum(),2))

O total de vendas do período é de: 785956.7


In [17]:
# estatística básica de valores de pedido

df_vendas['Valor_Pedido'].describe()

count    9088.000000
mean       86.482912
std        94.828346
min         1.500000
25%        28.987500
50%        55.490000
75%       110.000000
max      1270.490000
Name: Valor_Pedido, dtype: float64

In [18]:
# Quantidade de pedidos
print("Total de pedidos:",df_vendas['ID_Pedido'].value_counts().sum())

Total de pedidos: 9088


In [19]:
# Verificando dados duplicados

df_vendas.duplicated().sum()

np.int64(0)

In [20]:
# Verificando dados nulos
df_vendas.isnull().sum()

Ano                 0
Mes                 0
ID_Pedido           0
ID_Cliente          0
Status_Pedido       0
Data_Pedido         0
Data_Envio          0
Valor_Pedido        0
Quantidade_Itens    0
Estado              0
Cidade              0
Pais                0
postal_code         0
dtype: int64

In [184]:
# o mesmo traatamento de dados foi realizado nos demais datasets

df_pub.head(5)

,Ano,Mes,Data_Acesso,ID,Cidade,Estado,Origem,Tipo_Evento,CEP
0,2024,4,2024-04-07,402,São Paulo,São Paulo,Organic,cancel,02675-031
1,2024,1,2024-01-23,402,São Paulo,São Paulo,Organic,cancel,02675-031
2,2024,6,2024-06-11,10,São Paulo,São Paulo,Email,cancel,08285-060
3,2024,2,2024-02-24,24,Guarujá,São Paulo,YouTube,cancel,11432-501
4,2024,2,2024-02-21,18,Praia Grande,São Paulo,Adwords,cancel,11717-260


In [185]:
# Criando dataframe dimensão regional

df_regiao = df_pub[["Cidade","Estado",'CEP']]
df_regiao.drop_duplicates()

,Cidade,Estado,CEP
0,São Paulo,São Paulo,02675-031
2,São Paulo,São Paulo,08285-060
3,Guarujá,São Paulo,11432-501
4,Praia Grande,São Paulo,11717-260
5,Paulínia,São Paulo,13140-000
...,...,...,...
14729,Viana,Maranhão,65215-000
16257,Canoinhas,Santa Catarina,89460-000
16546,Belo Horizonte,Minas Gerais,31270-705
19376,Bodocó,Pernambuco,56220-000


In [186]:
# Gerando um código de calendário com auxílio do chat gpt hehehe

# Definir os anos que queremos gerar
anos = [2020, 2021, 2022, 2023, 2024]

# Lista para armazenar os dados do calendário
lista_calendario = []

# Mapeamento dos nomes dos meses em português
meses_pt = [
    '',
    'Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho',
    'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro'
]

# Iterar sobre cada ano
for ano in anos:
    # Iterar sobre cada mês
    for mes in range(1, 13):
        # Obter o nome do mês em português
        nome_mes = meses_pt[mes]
        
        # Obter o número de dias no mês
        dias_no_mes = calendar.monthrange(ano, mes)[1]
        
        # Iterar sobre cada dia no mês
        for dia in range(1, dias_no_mes + 1):
            # Obter o dia da semana (segunda-feira = 0, domingo = 6)
            dia_semana = calendar.day_name[calendar.weekday(ano, mes, dia)]
            
            # Formatar a data no formato dia/mês/ano
            data_formatada = f"{dia:02}/{mes:02}/{ano}"
            
            # Adicionar as informações ao dicionário
            info_dia = {
                'Ano': ano,
                'Mês': mes,
                'Nome do Mês': nome_mes,
                'Número do Dia': dia,
                'Data': data_formatada
            }
            
            # Adicionar o dicionário à lista
            lista_calendario.append(info_dia)

# Criar o DataFrame a partir da lista de dicionários
df_calendario = pd.DataFrame(lista_calendario)

#### Carga de Dados

In [1]:
# Salvando dataframes em armazenamento local para carga no tableau

dir_vendas = "C:/Users/User/Documents/Projeto_Google_Big_Query/datasets/df_vendas.csv"
dir_pub = "C:/Users/User/Documents/Projeto_Google_Big_Query/datasets/df_pub.csv"
dir_reg = "C:/Users/User/Documents/Projeto_Google_Big_Query/datasets/df_regiao.csv"
dir_cli = "C:/Users/User/Documents/Projeto_Google_Big_Query/datasets/df_cliente.csv"
dir_cal = "C:/Users/User/Documents/Projeto_Google_Big_Query/datasets/df_calendario.csv"
df_vendas.to_csv(dir_vendas, index = False)
df_pub.to_csv(dir_pub, index = False)
df_regiao.to_csv(dir_reg, index = False)
df_cliente.to_csv(dir_cli, index = False)
df_calendario.to_csv(dir_cal, index = False)

NameError: name 'df_vendas' is not defined

### BI de Vendas no Tableau

In [2]:
print("https://public.tableau.com/app/profile/george.feitosa/viz/TheLookEcommerceGBQ/AnlisedeFaturamento")

https://public.tableau.com/app/profile/george.feitosa/viz/TheLookEcommerceGBQ/AnlisedeFaturamento
